In [7]:
%pip install earthengine-api --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.1/256.1 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for google-cloud-storage from https://files.pythonhosted.org/packages/68/b8/cb00819641313e67b640857ffff2c5afdcdcfb4940def9f1502bf614a6d9/google_cloud_storage-2.12.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for google-api-python-client>=1.12.1 from https://files.pythonhosted.org/packages/0b/9a/69b094da2ddce819b905e3d63e3bf8713a519a5cec0b109f39a1e5da8926/google_api_python_client-2.105.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for google-auth>=1.4.1 from https://files.pythonhosted.org/packages/39/7c/2e4fa55a99f83ef9ef229ac5d59c44ceb90e2d0145711590c0fa39669f32/google_auth-2.23.3-py2.py3-none-any.whl.metadata
  Obtaining dependency information for google-auth-httplib2>=0.0.3 from https://files.pythonhosted.org/packages/d3/3d/e4991229886c0d522d9552151a43ff7adcc61e026e60ce8bd508387f84cf/goo

In [8]:
import ee
import numpy as np
import pandas as pd

In [9]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code:  4/1AfJohXk3zKRpFT16_7uRLJcOCQS-8H3nuPyemm9GRXOJ7cpO-dmQ0BrdAM8



Successfully saved authorization token.


In [12]:
dataset = ee.ImageCollection('NASA/SMAP/SPL4SMGP/007').filter(ee.Filter.date('2017-04-01', '2017-04-30'))

In [16]:
dataset.select(["snow_mass", "surface_temperature"])